# **Particle Swarm optimization**
class Particle:
- Define the particle object with the **__init__**. Every particle must have a position, a velocity.
- Define a function, called **FitnessCalculator**, that computes the fitness value of the particle. Its indipendent variable is the **accuracy metric** of the neural network.
- Define a function, called **PositionCalculator**
- Define a function, called **VelocityCalculator**


In [ ]:
import numpy as np

In [34]:
class Particle:

    def __init__(self, position, velocity ):
        '''position and velocity are vectors (lists)'''

        self.position = position
        self.velocity = velocity
        self.bestp = position
        # I need to inizialize the best position and also the fitness related to the best position
        # self.bestfitness = self.FitnessCalculator(self.bestp, model)

    def GetPosition(self):

        return self.position
    
    def GetVelocity(self):

        return self.velocity

    def FitnessCalculator(self, parameter, model):
        '''It takes as input the model and the parameter (which isthe particle position).
         Calculates the accuracy (or loss, we need to decide) of the model and return it. '''
        # Ho messo model.accuracy, poi quando definiamo il modello credo qui vada messa la parte proprio del training per ritornare poi l'accuracy o la loss in base a come vogliamo fare noi.
        self.fitness = model(parameter).accuracy
        return self


    def VelocityCalculator(self, w, c1, c2, random_1, random_2, best_glob_pos):

        self.velocity = w*self.velocity + c1*random_1*(self.bestp - self.position) + c2*random_2*(best_glob_pos - self.position)


    def PositionCalculator(self, new_vel):

        for dimension in range(len(self.position)):
            self.position[dimension] = self.position[dimension] + new_vel[dimension]

        self.FitnessCalculator(self, self.position, model)
        # Every time we update the positioon, we check also in the best personal position should be updated
        if self.bestfitness > self.fitness:
            self.bestp = self.position
            self.bestfitness = self.fitness



In [35]:
import random

def InitializeSwarm(swarm_size, dimensionality):

    swarm_list = []
    for particle in range(swarm_size):
        position = []
        velocity = []
        for dimension in dimensionality:
            position.append(random.random())
            velocity.append(random.random())

        part = Particle(position, velocity)
        part_pos = part.GetPosition()
        part_vel = part.GetVelocity()
        swarm_list.append([part_pos, part_vel])
        # e se invece facessimo:
        # swarm_list.append(part)

    return swarm_list


def EvaluateFitness(foo):

    return foo


def PsoAlgorithm(swarm_size, dimensionality, optimal_value):

    #first of all we initialize the swarm: it will be a list of particles
    swarm_list = InitializeSwarm(swarm_size, dimensionality)

    #we run the neural network for every particle in the swarm list: the neural netw function
    #returns the fitness function that the particle has reached
    termination_criteria = False
    first_cost = random.randrange(0, 10)
    second_cost = random.randrange(0, 10)
    best_locals_pos = [particle[0] for particle in swarm_list]
    best_global_pos_index = best_locals_pos.index(max(EvaluateFitness(best_locals_pos)))  #ATTENZIONE: DIPENDE DA COME ANDRò A DEFINIRE EvaluateFitness
    # quindi voglio che EvaluateFitness agisca su una lista di posizioni: quindi una lista di liste d-dim
    best_global_pos = swarm_list[best_global_pos_index][0]
    while termination_criteria == False:
        '''fitness_list = []
        #best_locals_pos = [particle[0] for particle in swarm_list]
        for particle in range(len(swarm_list)):
            fitness_list.append(EvaluateFitness(swarm_list[particle][0]))   #the fitness value will be dependent from particle position
        
        
        index_best_global = fitness_list.index(max(fitness_list))  #sarà l'indice della particella la cui posizione è bestglobal
        best_global_pos = swarm_list[index_best_global][0]  #swarm_list[index_best_global][0] è la posizione del bestglobal che a sua volta è un vettore d-dim'''
        fitness_list = []
        if max(fitness_list) > optimal_value:
            termination_criteria == True

        unif_rand_1 = []
        unif_rand_2 = []
        for dimension in range(dimensionality):
            unif_rand_1.append(random.uniform(0, 1))
            unif_rand_2.append(random.uniform(0, 1))
        
        for particle in range(len(swarm_list)):  #mo calcolo i nuovi 
        #for dimension in range(dimensionality):
            part = Particle(swarm_list[particle][0], swarm_list[particle][1])
            swarm_list[particle][1] = part.VelocityCalculator(first_cost = first_cost, second_cost = second_cost, random_1= 
                                                                unif_rand_1, random_2= unif_rand_2, best_loc_pos= best_locals_pos, 
                                                                best_glob_pos= best_global_pos)
            swarm_list[particle][0] = part.PositionCalculator(swarm_list[particle][1])
            if swarm_list[particle][]



In [1]:
class pepi:
    def __init__(self, position):
        self.position = position

    def getPosition(self):
        return self.position
    
mamam = pepi([2,6])
print(mamam.getPosition())

[2, 6]


In [10]:
a = 10
print([i**2 for i in [0,1,2,3]])

[0, 1, 4, 9]
